open the following link with a new tab

<a href="https://colab.research.google.com/github/nzhinusoftcm/review-on-collaborative-filtering/blob/master/6.NonNegativeMatrixFactorization.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Non-negative Matrix Factorization for Recommendations

Jusl like Matrix Factorization (MF) [(Yehuda Koren et al., 2009)](https://ieeexplore.ieee.org/document/5197422), Non-negative Matrix Factorization (NMF in short) factors the rating matrix $R$ in two matrices in such a way that $R = PQ^{\top}$.

### One limitation of Matrix Factorization

$P$ and $Q$ values in MF are non interpretable since their components can take arbitrary (positive and negative) values.

### Particulariy of Non-negative Matrix Factorization

NMF [(Lee and Seung, 1999)](http://www.dm.unibo.it/~simoncin/nmfconverge.pdf) allows the reconstruction of $P$ and $Q$ in such a way that $P,Q \ge 0$. Constrain $P$ and $Q$ values to be taken from $[0,1]$ allows  a probabilistic interpretation

- Latent factors represent groups of users who share the same tastes,
- The value  $P_{u,l}$  represents the probability that user $u$ belongs to the group $l$ of users and 
- The value $Q_{l,i}$ represents the probability that users in the group $l$  likes item $i$.

### Objective function

With the Euclidian distance, the NMF objective function is defined by

\begin{equation}
J = \frac{1}{2}\sum_{(u,i) \in \kappa}||R_{u,i} - P_uQ_i^{\top}||^2 + \lambda_P||P_u||^2 + \lambda_Q||Q_i||^2
\end{equation}

The goal is to minimize the cost function $J$ by optimizing parameters $P$ and $Q$, with $\lambda_P$ and $\lambda_Q$ the regularizer parameters.

### Multiplicative update rule

According [(Lee and Seung, 1999)](https://www.nature.com/articles/44565.pdf), to the multiplicative update rule for $P$ and $Q$ are as follows :

\begin{equation}
P \leftarrow P \cdot \frac{RQ}{PQ^{\top}Q}
\end{equation}

\begin{equation}
Q \leftarrow Q \cdot \frac{R^{\top}P}{QP^{\top}P}
\end{equation}

However, since $R$ is a sparse matrix, we need to update each $P_u$ according to existing ratings of user $u$. Similarily, we need to update $Q_i$ according to existing ratings on item $i$. Hence :

\begin{equation}
P_{u,k} \leftarrow P_{u,k} \cdot \frac{\sum_{i \in I_u}Q_{i,k}\cdot r_{u,i}}{\sum_{i \in I_u}Q_{i,k}\cdot \hat{r}_{u,i} + \lambda_P|I_u|P_{u,k}}
\end{equation}

\begin{equation}
Q_{i,k} \leftarrow Q_{i,k} \cdot \frac{\sum_{u \in U_i}P_{u,k}\cdot r_{u,i}}{\sum_{u \in U_i}P_{u,k}\cdot \hat{r}_{u,i} + \lambda_Q|U_i|Q_{i,k}}
\end{equation}

Where
- $P_{u,k}$ is the $k^{th}$ latent factor of $P_u$
- $Q_{i,k}$ is the $k^{th}$ latent factor of $Q_i$
- $I_u$ the of items rated by user $u$
- $U_i$ the set of users who rated item $i$

In [ ]:
import os

if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

### Install and import useful packages

### requirements

```
matplotlib==3.2.2
numpy==1.18.1
pandas==1.0.5
python==3.6.10
scikit-learn==0.23.1
scipy==1.5.0
```

In [1]:
from recsys.preprocessing import mean_ratings
from recsys.preprocessing import normalized_ratings
from recsys.preprocessing import  encode_data
from recsys.preprocessing import split_data
from recsys.preprocessing import rating_matrix
from recsys.preprocessing import get_examples
from recsys.preprocessing import scale_ratings

from recsys.datasets import mlLastedSmall
from recsys.datasets import ml1m
from recsys.datasets import ml100k

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

### Load and preprocess rating

In [2]:
# load data
ratings, movies = mlLastedSmall.load()

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column="rating")

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data(ratings, examples=examples, labels=labels)

In [3]:
ratings.userid = uencoder.transform(ratings.userid.to_list())
ratings.itemid = iencoder.transform(ratings.itemid.to_list())

### Non-negative Matrix Factorization Model

In [4]:
class NonNegativeMF:
    
    def __init__(self, ratings, m, n, K=10, lambda_P=0.01, lambda_Q=0.01):
        
        np.random.seed(32)
        
        # initialize the latent factor matrices P and Q (of shapes (m,k) and (n,k) respectively) that will be learnt
        self.ratings = ratings
        self.K = K
        self.P = np.random.rand(m, K)
        self.Q = np.random.rand(n, K)
        
        # hyperparameter initialization
        self.lambda_P = lambda_P
        self.lambda_Q = lambda_Q
        
        # training history
        self.history = {
            "epochs":[],
            "loss":[],
            "val_loss":[],
        }
    
    def print_training_parameters(self):
        print('Training Matrix Factorization Model ...')
        print(f'k={self.K}')
        
    def mae(self,  x_train, y_train):
        """
        returns the Mean Absolute Error
        """
        # number of training exemples
        M = x_train.shape[0]
        error = 0
        for pair, r in zip(x_train, y_train):
            u, i = pair
            error += abs(r - np.dot(self.P[u], self.Q[i]))
        return error/M
    
    def ratings_by_this_user(self, userid):
        return self.ratings.loc[self.ratings.userid==userid]
    
    def ratings_on_this_items(self, itemid):
        return self.ratings.loc[self.ratings.itemid==itemid]
    
    def update_rule(self, u, i, error):
        I = self.ratings_by_this_user(u)
        U = self.ratings_on_this_items(i)    
        
        for k in range(self.K):
            
            num_uk = self.P[u,k] * np.sum(np.multiply(self.Q[I.itemid.to_list(),k], I.rating.to_list()))
            dem_uk = np.sum(np.multiply(
                self.Q[I.itemid.to_list(),k], 
                np.dot(self.P[u], self.Q[I.itemid.to_list()].T))
                           ) + self.lambda_P * len(I) * self.P[u,k]
            self.P[u,k] = num_uk / dem_uk
                
            num_ik = self.Q[i,k] * np.sum(np.multiply(self.P[U.userid.to_list(),k], U.rating.to_list()))
            dem_ik = np.sum(np.multiply(
                self.P[U.userid.to_list(),k], 
                np.dot(self.P[U.userid.to_list()], self.Q[i].T))
                           ) + self.lambda_Q * len(U) * self.Q[i,k]
            self.Q[i,k] = num_ik / dem_ik
                
    
    def print_training_progress(self, epoch, epochs, error, val_error, steps=5):
        if epoch == 1 or epoch % steps == 0 :
                print("epoch {}/{} - loss : {} - val_loss : {}".format(
                    epoch, epochs, round(error,3), round(val_error,3)))
                
    def fit(self, x_train, y_train, validation_data, epochs=10):

        self.print_training_parameters()
        x_test, y_test = validation_data
        for epoch in range(1, epochs+1):
            for pair, r in zip(x_train, y_train):
                u,i = pair
                r_hat = np.dot(self.P[u], self.Q[i])
                e = abs(r - r_hat)
                self.update_rule(u, i, e)                
            # training and validation error  after this epochs
            error = self.mae(x_train, y_train)
            val_error = self.mae(x_test, y_test)
            self.update_history(epoch, error, val_error)
            self.print_training_progress(epoch, epochs, error, val_error, steps=1)
        
        return self.history
    
    def update_history(self, epoch, error, val_error):
        self.history['epochs'].append(epoch)
        self.history['loss'].append(error)
        self.history['val_loss'].append(val_error)
    
    def evaluate(self, x_test, y_test):        
        error = self.mae(x_test, y_test)
        print(f"validation error : {round(error,3)}")
        print('MAE : ', error)        
        return error
      
    def predict(self, userid, itemid):
        u = uencoder.transform([userid])[0]
        i = iencoder.transform([itemid])[0]
        r = np.dot(self.P[u], self.Q[i])
        return r

    def recommend(self, userid, N=30):
        # encode the userid
        u = uencoder.transform([userid])[0]

        # predictions for users userid on all product
        predictions = np.dot(self.P[u], self.Q.T)

        # get the indices of the top N predictions
        top_idx = np.flip(np.argsort(predictions))[:N]

        # decode indices to get their corresponding itemids
        top_items = iencoder.inverse_transform(top_idx)

        # take corresponding predictions for top N indices
        preds = predictions[top_idx]

        return top_items, preds   

In [5]:
m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

# create and train the model
NMF = NonNegativeMF(ratings, m, n, K=10,  lambda_P=0.5, lambda_Q=0.5)
history = NMF.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10
epoch 1/10 - loss : 0.779 - val_loss : 0.801
epoch 2/10 - loss : 0.775 - val_loss : 0.797
epoch 3/10 - loss : 0.773 - val_loss : 0.796
epoch 4/10 - loss : 0.772 - val_loss : 0.795
epoch 5/10 - loss : 0.771 - val_loss : 0.795
epoch 6/10 - loss : 0.771 - val_loss : 0.795
epoch 7/10 - loss : 0.77 - val_loss : 0.795
epoch 8/10 - loss : 0.77 - val_loss : 0.795
epoch 9/10 - loss : 0.77 - val_loss : 0.794
epoch 10/10 - loss : 0.769 - val_loss : 0.794


In [6]:
NMF.evaluate(X_test, y_test)

validation error : 0.794
MAE :  0.7943610363378308


0.7943610363378308

## Evaluate NMF on ML100k dataset

In [7]:
# load data
ratings, movies = ml100k.load()

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column="rating")

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data(ratings, examples=examples, labels=labels)

In [8]:
ratings.userid = uencoder.transform(ratings.userid.to_list())
ratings.itemid = iencoder.transform(ratings.itemid.to_list())

In [9]:
m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

# create and train the model
NMF = NonNegativeMF(ratings, m, n, K=10,  lambda_P=0.5, lambda_Q=0.5)
history = NMF.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10
epoch 1/10 - loss : 0.866 - val_loss : 0.865
epoch 2/10 - loss : 0.866 - val_loss : 0.865
epoch 3/10 - loss : 0.866 - val_loss : 0.865
epoch 4/10 - loss : 0.866 - val_loss : 0.865
epoch 5/10 - loss : 0.866 - val_loss : 0.865
epoch 6/10 - loss : 0.866 - val_loss : 0.865
epoch 7/10 - loss : 0.866 - val_loss : 0.865
epoch 8/10 - loss : 0.866 - val_loss : 0.865
epoch 9/10 - loss : 0.866 - val_loss : 0.865
epoch 10/10 - loss : 0.866 - val_loss : 0.865


In [10]:
NMF.evaluate(X_test, y_test)

validation error : 0.865
MAE :  0.8647139758177113


0.8647139758177113

## Reference

1. Daniel D. Lee & H. Sebastian Seung (1999). [Learning the parts of objects by non-negative matrix factorization](https://www.nature.com/articles/44565)
2. Deng Cai et al. (2008). [Non-negative Matrix Factorization on Manifold](https://ieeexplore.ieee.org/document/4781101)
3. Yu-Xiong Wang and Yu-Jin Zhang (2011). [Non-negative Matrix Factorization: a Comprehensive Review](https://ieeexplore.ieee.org/document/6165290)
4. Nicolas Gillis (2014). [The Why and How of Nonnegative Matrix Factorization](https://arxiv.org/pdf/1401.5226.pdf)

## Author

<a href="https://www.linkedin.com/in/carmel-wenga-871876178/">Carmel WENGA</a>, Applied Machine Learning Research Engineer | <a href="https://shoppinglist.cm/fr/">ShoppingList</a>, Nzhinusoft